<a href="https://colab.research.google.com/github/MiguelAngel-ht/Machiine_Learning_Models/blob/main/CHAT_BOT_Training_with_Sequential_Algorithm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CREAT JSON WITH THIS FORM TO DATA TRAINING

x = {"intents":[
    {"tag": "saludos",
     "patterns": ["hola", "hi", "hello", "kiubo", "que onda", "oli",
                  "oliwis", "hey", "que pdo", "kiubole", "ola", "que tranza"],
     "responses": ["Hola", "¡Qué tal!", "¡Qué onda!"]
     },
    {"tag": "adios",
     "patterns": ["adios", "adiós", "bye", "ahí la ves", "ay la bimbo", 
                  "paro carnal", "hay nos wachamos", "nos vemos", "hasta luego",
                  "hasta mañana", "buenas noches", "bonita tarde"],
     "responses": ["¡Adiós, Hasta luego!", "¡Buen día! ¡Nos vemos!", 
                   "¡Qué le vaya bien!", "¡Hasta pronto wapx!"]
     }
]}

In [2]:
# LIBRARIES 

import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [7]:
lemmatizer = WordNetLemmatizer()  # initialization

intents = json.loads(open('intents.json').read())
# intetnts = json.loads(x)  # DIRECT DEFITION IN THE SAME CODE

words = []
classes = []
documents = []
ignore_letters = ["?","¿","¡","!",",",".",";",":","-"]

In [8]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [9]:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [10]:
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [11]:
training = []
output_empty = [0] * len(classes)

In [12]:
for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training,dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [13]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))                     # Dropout(0,5) => Dropout(0.5)
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)  # lr is deprecated
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [14]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
model.save('chatbot_model.h5',hist)
print('Done')

Epoch 1/50
5/5 [==============================] - 1s 4ms/step - loss: 0.7348 - accuracy: 0.5000
Epoch 2/50
5/5 [==============================] - 0s 4ms/step - loss: 0.7234 - accuracy: 0.5000
Epoch 3/50
5/5 [==============================] - 0s 4ms/step - loss: 0.6901 - accuracy: 0.5833
Epoch 4/50
5/5 [==============================] - 0s 4ms/step - loss: 0.6685 - accuracy: 0.5417
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: 0.6279 - accuracy: 0.7500
Epoch 6/50
5/5 [==============================] - 0s 4ms/step - loss: 0.6978 - accuracy: 0.4583
Epoch 7/50
5/5 [==============================] - 0s 4ms/step - loss: 0.5958 - accuracy: 0.7917
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.6177 - accuracy: 0.6667
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.6198 - accuracy: 0.7083
Epoch 10/50
5/5 [==============================] - 0s 4ms/step - loss: 0.5604 - accuracy: 0.9167
Epoch 11/50
5/5 [======================

In [15]:
from tensorflow.keras.models import load_model

In [16]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')

In [17]:
def clean_up_Sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [18]:
def bag_of_words(sentence):
  sentence_words = clean_up_Sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

In [19]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_TRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r > ERROR_TRESHOLD]
  
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
  return return_list

In [20]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  result = ''
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [30]:
res = ""
print('Escribir 0 para finalizar \n')

while True:
  message = input("Tú: ")
  if message == '0': break
  ints = predict_class(message)
  res = get_response(ints, intents)
  print("Bot: ",res)

print("Bot: Gracias por hablar conmigo! >///<")

Escribir 0 para finalizar 

Tú: hi
Bot:  ¡Qué onda!
Tú: Hola
Bot:  ¡Qué tal!
Tú: que pdo
Bot:  ¡Qué tal!
Tú: adios
Bot:  ¡Buen día! ¡Nos vemos!
Tú: bye
Bot:  ¡Buen día! ¡Nos vemos!
Tú: 0
Bot: Gracias por hablar conmigo! >///<
